<a href="https://colab.research.google.com/github/hellocybernetics/TensorFlow_Eager_Execution_Tutorials/blob/master/tutorials/02_intermediate/Convolutinal_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()
tfe = tf.contrib.eager
L = tf.keras.layers

In [3]:
# Hyper parameters
num_epochs = 10
num_classes = 10
batch_size = 1024
learning_rate = 0.001

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print("training_data\n", x_train.shape)
print("test_data\n", x_test.shape)
print("training_label\n", y_train.shape)
print("test_label\n", y_test.shape)

170500096/170498071 [==============================] - 36s 0us/step
training_data
 (50000, 32, 32, 3)
test_data
 (10000, 32, 32, 3)
training_label
 (50000, 1)
test_label
 (10000, 1)


In [4]:
x_train_ = tf.transpose(tf.convert_to_tensor(x_train, dtype=tf.float32), 
                        [0, 3, 1, 2])
y_train_ = tf.reshape(tf.one_hot(y_train, 10), (-1, 10))

print(x_train_.shape)
print(y_train_.shape)

(50000, 3, 32, 32)
(50000, 10)


In [5]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .batch(batch_size)
    .shuffle(10000)
)

train_dataset = (
    train_dataset.map(lambda x, y: 
                      (tf.div(tf.cast(
                          tf.transpose(x, [0, 3, 1, 2]), tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)

print(train_dataset)

<MapDataset shapes: ((?, 3, 32, 32), (?, 10)), types: (tf.float32, tf.float32)>


In [6]:
test_dataset = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .batch(1000)
    .shuffle(10000)
)
test_dataset = (
    test_dataset.map(lambda x, y: 
                      (tf.div(tf.cast(
                          tf.transpose(x, [0, 3, 1, 2]), tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)
print(test_dataset)

<MapDataset shapes: ((?, 3, 32, 32), (?, 10)), types: (tf.float32, tf.float32)>


In [0]:
class Cifar10Model(tf.keras.Model):
    def __init__(self):
        super(Cifar10Model, self).__init__(name='cifar_cnn')
        
        self.conv_block1 = tf.keras.Sequential([
            L.Conv2D(
                8, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_initializer=tf.initializers.variance_scaling,
                kernel_regularizer=tf.keras.regularizers.l2(l=0.001),
                data_format="channels_first"
            ),
            L.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
                data_format="channels_first"
            ),
            L.BatchNormalization(axis=1),
        ])

        self.conv_block2 = tf.keras.Sequential([
            L.Conv2D(
                16, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_initializer=tf.initializers.variance_scaling,
                kernel_regularizer=tf.keras.regularizers.l2(l=0.001),
                data_format="channels_first"
            ),
            L.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
                data_format="channels_first"
            ),
            L.BatchNormalization(axis=1),
        ])
        
        self.conv_block3 = tf.keras.Sequential([
            L.Conv2D(
                32, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_initializer=tf.initializers.variance_scaling,
                kernel_regularizer=tf.keras.regularizers.l2(l=0.001),
                data_format="channels_first"
            ),
            L.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
                data_format="channels_first"
            ),
            L.BatchNormalization(axis=1),
        ])
        
        self.flatten = L.Flatten()
        self.fc1 = L.Dense(
            256, 
            activation=tf.nn.relu,
            kernel_initializer=tf.initializers.variance_scaling,
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001))
        self.dropout = L.Dropout(0.8)
        self.fc2 = L.Dense(10)
        self.softmax = L.Softmax()

    def call(self, x, training=False):
        x = self.conv_block1(x, training=training)
        x = self.conv_block2(x, training=training)
        x = self.conv_block3(x, training=training)
        x = self.flatten(x)
        x = self.dropout(self.fc1(x), training=training)
        x = self.fc2(x)
        return self.softmax(x)

In [0]:
model = Cifar10Model()

def loss_fn(y, y_pre):
    return tf.keras.losses.categorical_crossentropy(y, y_pre)

def accuracy(y, y_pre):
    return tf.keras.metrics.categorical_accuracy(y, y_pre)

optimizer = tf.train.AdamOptimizer(learning_rate)

In [23]:
for j in range(num_epochs):
    
    running_loss = 0
    running_acc = 0

    for i, (x_, y_) in enumerate(train_dataset):
        
        with tf.device("/gpu:0"):
            with tf.GradientTape() as tape:
                y_pre = model(x_, training=True)
                loss = loss_fn(y_, y_pre)
            acc = accuracy(y_, y_pre)
            grads = tape.gradient(loss, model.variables)
            optimizer.apply_gradients(zip(grads, model.variables))
            running_loss += tf.reduce_mean(loss)
            running_acc += tf.reduce_mean(acc)
    
    print("-----epoch {} -----".format(j + 1))
    print("loss: ", running_loss.numpy()/(i + 1))
    print("acc: ", running_acc.numpy()/(i + 1))    

-----epoch 1 -----
loss:  2.080876720194914
acc:  0.27564558690908003
-----epoch 2 -----
loss:  1.6930190105827487
acc:  0.38147042722118146
-----epoch 3 -----
loss:  1.5609390881596779
acc:  0.43662760209064094
-----epoch 4 -----
loss:  1.4644119885503029
acc:  0.4703996619399713
-----epoch 5 -----
loss:  1.3926103552993463
acc:  0.4998274044114716
-----epoch 6 -----
loss:  1.323324086714764
acc:  0.5280563198790258
-----epoch 7 -----
loss:  1.2741163604113521
acc:  0.5484468109753667
-----epoch 8 -----
loss:  1.2303034724021444
acc:  0.5626135845573581
-----epoch 9 -----
loss:  1.1981804516850685
acc:  0.5748023597561583
-----epoch 10 -----
loss:  1.1650082335180165
acc:  0.5883636863864198


In [24]:
test_accuracy = 0
for i, (x_, y_) in enumerate(test_dataset):
    y_pre = model(x_)
    test_accuracy += tf.reduce_mean(accuracy(y_, y_pre))
test_accuracy /= i + 1

print("test accuracy {:0.3f}".format(test_accuracy.numpy()))

test accuracy 0.623
